In [4]:
from datetime import datetime
import MetaTrader5 as mt5 
# display data on the MetaTrader 5 package 
print("MetaTrader5 package author: ",mt5.__author__) 
print("MetaTrader5 package version: ",mt5.__version__) 
  
# import the 'pandas' module for displaying data obtained in the tabular form
import pandas as pd
pd.set_option('display.max_columns', 500) # number of columns to be displayed
pd.set_option('display.width', 30000)      # max table width to display
# import pytz module for working with time zone
import pytz 
import talib as ta
import math as ma
import numpy as np

# establish connection to MetaTrader 5 terminal 
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error()) 
    quit()
 
# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")

print(timezone)
# create 'datetime' objects in UTC time zone to avoid the implementation of 
# a local time zone offset

utc_from = datetime(2020, 9, 5, tzinfo=timezone)
utc_fromM1 = datetime(2020, 9, 5, tzinfo=timezone)
utc_fromM5= datetime(2020, 9, 5, tzinfo=timezone)
utc_fromM15 = datetime(2020, 9, 5, tzinfo=timezone)
utc_fromM30 = datetime(2020, 9, 5, tzinfo=timezone)
utc_fromH1 = datetime(2020, 9, 5, tzinfo=timezone)
utc_fromH4 = datetime(2020, 9, 5, tzinfo=timezone)
utc_fromD1 = datetime(2020, 9, 5, tzinfo=timezone)
utc_from = datetime(2020, 9, 5, tzinfo=timezone)
utc_to = datetime(2020, 9, 15, hour = 23, minute = 59,microsecond = 999999,tzinfo=timezone)

utc_current = datetime.now()

UTC = pytz.utc
utc_current_UTC = datetime.now(UTC)

print(utc_from)
print(utc_to)
print(utc_current)
print(utc_current_UTC)

# get bars from USDJPY M5 within the interval of 2020.01.10 00:00 - 2020.01.11 13:00 
# in UTC time zone
rates = mt5.copy_rates_range("GOLD", mt5.TIMEFRAME_M1, utc_from, utc_to)
 
# shut down connection to the MetaTrader 5 terminal
mt5.shutdown() 

# display each element of obtained data in a new line
print("Display obtained data 'as is'")
counter=0
for rate in rates:
    counter+=1
    if counter<=10:
        print(rate)
 
# create DataFrame out of the obtained data
rates_frame = pd.DataFrame(rates)
# convert time in seconds into the 'datetime' format
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
rates_frame['ATR']=ta.ATR(rates_frame['high'],rates_frame['low'],rates_frame['close'],timeperiod=1)

from talib import MA_Type

rates_frame['macd'], rates_frame['macdsignal'], rates_frame['macdhist'] = ta.MACD(
                                rates_frame['close'], 
                                fastperiod=8, 
                                slowperiod=17, 
                                signalperiod=9)

rates_frame['upper'], rates_frame['middle'], rates_frame['lower'] = ta.BBANDS(
                                rates_frame['close'], 
                                timeperiod=20,
                                # number of non-biased standard deviations from the mean
                                nbdevup=2,
                                nbdevdn=2,
                                # Moving average type: simple moving average here
                                matype=0)
rates_frame['williams'] = ta.WILLR(
                                rates_frame['high'],
                                rates_frame['low'],
                                rates_frame['close'],
                                timeperiod=14)
rates_frame['BB_D']=rates_frame['upper'] -rates_frame['lower']
rates_frame['BB_S']=rates_frame['BB_D'] -rates_frame['BB_D'].shift(1)

rates_frame['result']='A'
rates_frame['result1']='B'

#print(rates_frame.dtypes)
print(rates_frame.count())
print('-----',rates_frame.BB_D.count())
rates_frame[rates_frame.BB_D.isnull()]

"""
counter=0
for rate in range(25,0,-1):
    counter+=1
    if counter<=21:
        print(rates_frame.macd.loc[rate])
#print(rates_frame.BB_D.loc[7371])
"""
rates_frameAll=rates_frame.time.count()-1
print('******',rates_frame.macd.loc[rates_frameAll])
#macd0 = rates_frame.macd.loc[1111]
#print(macd0)

for i in range(rates_frameAll,2,-1):
    macd0 = rates_frame.macd.loc[i]
    macdS1 = rates_frame.macd.loc[i-1]
    perR0 = rates_frame.williams.loc[i]
    perRS1 = rates_frame.williams.loc[i-1]
    if macd0 > 0:
        if macd0 < macdS1 :
            rates_frame.result.loc[i] = 'd'
        else:
            rates_frame.result.loc[i] = 'u'
    else:
        if macd0 < macdS1 :
            rates_frame.result.loc[i] = 'd'
        else:
            rates_frame.result.loc[i] = 'u'
    rates_frame.williams.loc[i] = perR0 - perRS1
counter=0
for rate in range(rates_frameAll,2,-1):
    counter+=1
    priceUse = round((rates_frame.open.loc[rate]+rates_frame.high.loc[rate]+rates_frame.low.loc[rate]+
                      rates_frame.close.loc[rate])/4,2)
    perRUse = round(rates_frame.williams.loc[rate],2)
    if counter<=40:
#print(rate,rates_frame.time.loc[rate],priceUse,'\t',rates_frame.macd.loc[rate],'\t',rates_frame.result.loc[rate])
        print(rate,rates_frame.time.loc[rate],priceUse,rates_frame.result.loc[rate],
        perRUse)
#print(rates_frame.dtypes)
#rates_frame[['time','macd','BB_D','BB_S']][6250:-1]
#print(rates_frame.tail(30))
# display data
#print("\nDisplay dataframe with data")
#print(rates_frame.head(10))
#print("\nDisplay dataframe with data") 

#rates_frame.BB_D.head()

#rates_frame1=pd.DataFrame(rates_frame)
#cols=['macd','output1']
#print(rates_frame1)

#print(rates_frame.describe())
#rates_frame.quantile([.25,.5,.75])
#print(rates_frame.max())
#rates_frame['ATR'].value_counts(bins=20,sort=False)
#rates_frame['diffBB'].value_counts(bins=20,sort=False)

MetaTrader5 package author:  MetaQuotes Software Corp.
MetaTrader5 package version:  5.0.33
Etc/UTC
2020-09-05 00:00:00+00:00
2020-09-15 23:59:00.999999+00:00
2020-10-05 16:57:10.667965
2020-10-05 09:57:10.668966+00:00
Display obtained data 'as is'
(1599440400, 1930.28, 1931.57, 1930.2, 1930.61, 39, 76, 0)
(1599440460, 1930.96, 1931.71, 1930.9, 1931.41, 34, 73, 0)
(1599440520, 1931.57, 1932., 1931.49, 1931.75, 27, 72, 0)
(1599440580, 1931.53, 1932.28, 1931.53, 1932.17, 31, 80, 0)
(1599440640, 1932.04, 1932.4, 1932., 1932., 19, 75, 0)
(1599440700, 1932.15, 1932.74, 1932.04, 1932.47, 32, 65, 0)
(1599440760, 1932.61, 1934.2, 1932.61, 1934.12, 35, 45, 0)
(1599440820, 1934.08, 1934.23, 1933.92, 1934.06, 32, 37, 0)
(1599440880, 1933.89, 1934.2, 1933.89, 1934.11, 27, 37, 0)
(1599440940, 1934.09, 1934.38, 1933.99, 1934.07, 22, 33, 0)
time           9408
open           9408
high           9408
low            9408
close          9408
tick_volume    9408
spread         9408
real_volume    9408
AT

In [3]:
#    if (rates_frame.macd.loc[i].isnull() | rates_frame.macd.loc[i-1].isnull() | rates_frame.macd.loc[i-2].isnull()):




for label,row in rates_frame.iterrows():
    
    if rates_frame['macd'].isnull() :
        print('ffff')
    else :
        print(rates_frame['macd'],rates_frame['macd'].shift(1))
        rates_X=rates_frame['macd'] < rates_frame['macd'].shift(1)
        rates_X1=rates_frame['macd'].shift(1) > rates_frame['macd'].shift(2)
    
        if rates_X == True :
            #    rates_frame['macd'].shift(1)>rates_frame['macd'].shift(2)):
            rates_frame['output2']='XD'

        else :
            rates_frame['output2']='NO'

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().